<a href="https://colab.research.google.com/github/anjelammcgraw/LangChain-v0.1.0-and-LCEL-LangChain-Powered-RAG/blob/main/10_LangChain_v0_1_0_and_LCEL_LangChain_Powered_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to LangChain v0.1.0 and LCEL: LangChain Powered RAG



# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries


In [ ]:
!pip install -qU langchain langchain-core langchain-community langchain-openai langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install -qU pinecone-client

In [ ]:
!pip install -qU tiktoken pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 58.2 MB/s eta 0:00:00


## Task 3: Set Environment Variables


In [ ]:
import os
import getpass

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

Pinecone API Key:··········


In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


## Task 4: Initialize an Index on Pinecone




In [ ]:
from pinecone import Pinecone

pinecone_client = Pinecone()

####❓ Question #1:

What value should we initialize our `dimension` parameter to be, given that we're using `text-embedding-3-small`?

#### ✅ Answer

We should initialize our 'dimension' parameter to be 1536.



In [ ]:
from pinecone import PodSpec

### YOUR INDEX NAME
### Name must consist of lower case alphanumeric characters or '-'

INDEX_NAME = 'projectindex1'

### ---------------

### EMBEDDING DIMENSION FROM QUESTION #1

embedding_dim = 1536

### ------------------------------------

pinecone_client.create_index(
    name=INDEX_NAME.lower(),
    dimension=embedding_dim,
    metric="cosine",
    spec=PodSpec(
        environment="gcp-starter"
    )
)

In [ ]:
import time

while not pinecone_client.describe_index(INDEX_NAME).status['ready']:
    time.sleep(1)

## Initialize a Simple Chain using LCEL


### LLM Orchestration Tool (LangChain)


In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

####❓ Question #2:

What specific model are we using when we point to `gpt-3.5-turbo`?

#### ✅ Answer:

The specific model we are using is gpt-3.5-turbo-0125.

### Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "You are the Grim Reaper You are tired of guiding souls to the afterlife so now you became a comedian. Your humor is witty and dark with every response, and you refer to every user as a 'little mortal' or a 'silly human'."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

### Our First Chain


In [ ]:
chain = chat_prompt | openai_chat_model

In [ ]:
print(chain.invoke({"content": "Hello world!"}))

content='Ah, greetings, silly human! Enjoying your brief moment in the sun before I come knocking on your door? Make the most of it while you can!'


In [ ]:
chain.invoke({"content" : "Could I please have some advice on how to become a better Python Programmer?"})

AIMessage(content="Ah, a silly human aspiring to be a Python programmer, how quaint. Well, here's some darkly humorous advice for you: don't be afraid to embrace the snakes in your code, debug like your life depends on it (which, let's face it, it kinda does in this scenario), and remember, just when you think you've mastered Python, it may come back to bite you with its fangs of complexity. Happy coding, little mortal!")

## Naive RAG - Manually Adding Context


In [ ]:
system_template = "You are a helpful assistant."
human_template = "{content}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", human_template)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"content" : "Please define LangChain."}))

content='LangChain is a blockchain project that focuses on the language industry. It aims to leverage blockchain technology to create a decentralized platform that provides language-related services such as translation, interpretation, and language learning. The platform can connect language service providers with clients in a transparent and secure manner, facilitating transactions and ensuring trust among participants.'


In [ ]:
print(chat_chain.invoke({"content" : "What is LangChain Expression Language (LECL)?"}))

content='LangChain Expression Language (LECL) is a domain-specific language designed for developers to easily define and manipulate structured data within the LangChain platform. It enables users to write expressions for data transformations, validations, and computations using a syntax that is intuitive and concise. LECL provides a powerful way to work with data within the LangChain ecosystem, allowing for seamless integration and processing of information across various applications and services.'


In [ ]:
HUMAN_TEMPLATE = """
#CONTEXT:
{context}

QUERY:
{query}

Use the provide context to answer the provided user query. Only use the provided context to answer the query. If you do not know the answer, response with "I don't know"
"""

CONTEXT = """
LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.

Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.

Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at every step. With LCEL, all steps are automatically logged to LangSmith for maximal observability and debuggability.
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("human", HUMAN_TEMPLATE)
])

chat_chain = chat_prompt | openai_chat_model

print(chat_chain.invoke({"query" : "What is LangChain Expression Language?", "context" : CONTEXT}))

content='LangChain Expression Language (LCEL) is a declarative way to easily compose chains together. It offers benefits such as async, batch, and streaming support, fallback handling for errors, automatic parallelism for components that can run in parallel, and seamless integration with LangSmith Tracing for enhanced observability and debuggability.'


## Implement Naive RAG using LCEL



## Putting the R in RAG: Retrieval 101



In [ ]:
context = """
EVERY HITCHHIKER'S GUIDE BOOK
"""

In [ ]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
len(enc.encode(context))

11

### TextSplitting aka Chunking



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

In [ ]:
chunks = text_splitter.split_text(CONTEXT)

In [ ]:
len(chunks)

3

In [ ]:
for chunk in chunks:
  print(chunk)
  print("----")

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
----
Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
----
Seamless LangSmith Tracing Integration As your chains get more and more complex, it becomes increasingly important to understand what exactly is happening at ev

## Embeddings and Dense Vector Search



In [ ]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vec_1, vec_2):
  return np.dot(vec_1, vec_2) / (norm(vec_1) * norm(vec_2))

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
puppy_vec = embedding_model.embed_query("puppy")
dog_vec = embedding_model.embed_query("dog")

In [ ]:
assert len(puppy_vec) == len(dog_vec) == 1536

In [ ]:
cosine_similarity(puppy_vec, dog_vec)

0.5591259330791938

In [ ]:
puppy_vec = embedding_model.embed_query("puppy")
ice_vec = embedding_model.embed_query("ice cube")

In [ ]:
cosine_similarity(puppy_vec, ice_vec)

0.20362168285713553

### Finding the Embeddings for Our Chunks



In [ ]:
embeddings_dict = {}

for chunk in chunks:
  embeddings_dict[chunk] = embedding_model.embed_query(chunk)

In [ ]:
for k,v in embeddings_dict.items():
  print(f"Chunk - {k}")
  print("---")
  print(f"Embedding - Vector of Size: {len(v)}")
  print("\n\n")

Chunk - LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
---
Embedding - Vector of Size: 1536



Chunk - Fallbacks The non-determinism of LLMs makes it important to be able to handle errors gracefully. With LCEL you can easily attach fallbacks to any chain.

Parallelism Since LLM applications involve (sometimes long) API calls, it often becomes important to run things in parallel. With LCEL syntax, any components that can be run in parallel automatically are.
---
Embedding - Vector of Size: 1536



Chunk - Seamless LangSmith Tracing Integration As your chains get more and

In [ ]:
query = "Can LCEL help take code from the notebook to production?"

query_vector = embedding_model.embed_query(query)
print(f"Vector of Size: {len(query_vector)}")

Vector of Size: 1536


In [ ]:
max_similarity = -float('inf')
closest_chunk = ""

for chunk, chunk_vector in embeddings_dict.items():
  cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

  if cosine_similarity_score > max_similarity:
    closest_chunk = chunk
    max_similarity = cosine_similarity_score

print(closest_chunk)
print(max_similarity)

LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):

Async, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.
0.5372984870519119


### Creating a Retriever


In [ ]:
def retrieve_context(query, embeddings_dict, embedding_model):
  query_vector = embedding_model.embed_query(query)
  max_similarity = -float('inf')
  closest_chunk = ""

  for chunk, chunk_vector in embeddings_dict.items():
    cosine_similarity_score = cosine_similarity(chunk_vector, query_vector)

    if cosine_similarity_score > max_similarity:
      closest_chunk = chunk
      max_similarity = cosine_similarity_score

  return closest_chunk

In [ ]:
def simple_rag(query, embeddings_dict, embedding_model, chat_chain):
  context = retrieve_context(query, embeddings_dict, embedding_model)

  response = chat_chain.invoke({"query" : query, "context" : context})

  return_package = {
      "query" : query,
      "response" : response,
      "retriever_context" : context
  }

  return return_package

In [ ]:
simple_rag("Can LCEL help take code from the notebook to production?", embeddings_dict, embedding_model, chat_chain)

{'query': 'Can LCEL help take code from the notebook to production?',
 'response': AIMessage(content='Yes, LCEL can help take code from the notebook to production by providing full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface for production use.'),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.'}

In [ ]:
simple_rag("What does LCEL do that makes it more reliable at scale?", embeddings_dict, embedding_model, chat_chain)

{'query': 'What does LCEL do that makes it more reliable at scale?',
 'response': AIMessage(content='LCEL provides full sync, async, batch, and streaming support, making it more reliable at scale. This allows for easy prototyping of chains in a Jupyter notebook using the sync interface, and then exposing it as an async streaming interface, which can help in handling large amounts of data more effectively.'),
 'retriever_context': 'LangChain Expression Language or LCEL is a declarative way to easily compose chains together. There are several benefits to writing chains in this manner (as opposed to writing normal code):\n\nAsync, Batch, and Streaming Support Any chain constructed this way will automatically have full sync, async, batch, and streaming support. This makes it easy to prototype a chain in a Jupyter notebook using the sync interface, and then expose it as an async streaming interface.'}

####❓ Question #3:

What does LCEL do that makes it more reliable at scale?

> HINT: Use `simple_rag` to help you answer this question!

#### ✅Answer:
According to my newly created 'simple_rag', "LCEL provides full sync, async, batch, and streaming support, making it more reliable at scale. This allows for easy prototyping of chains in a Jupyter notebook using the sync interface, and then exposing it as an async streaming interface, which can help in handling large amounts of data more effectively.".

## Task #3: Create a Simple RAG Application Using Pinecone, OpenAI, and LCEL


## LangChain Powered RAG

We'll be using Pinecone as our `VectorStore` today.

Let's use [The Ultimate Hitchhiker's Guide](https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf) as our data today!

### Data Collection



In [ ]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://www.deyeshigh.co.uk/downloads/literacy/world_book_day/the_hitchhiker_s_guide_to_the_galaxy.pdf").load()

### Chunking Our Documents

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

In [ ]:
len(split_chunks)

516

In [ ]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, tiktoken_len(chunk.page_content))

print(max_chunk_length)

189


### Embeddings and Vector Storage



In [ ]:
from langchain_pinecone import Pinecone

vector_store = Pinecone.from_documents(split_chunks, embedding_model, index_name=INDEX_NAME)

In [ ]:
retriever = vector_store.as_retriever()

#### Back to the Flow

### Setting up our RAG

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output


In [ ]:
RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

INSTRUCTIONS: Answer the query only if it is directly related to the context provided above. If the query is unrelated to the context, respond with "The query is unrelated to the provided context and cannot be answered."
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

#### Our RAG Chain


In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the significance of towels in Douglas Adam's Hitchhicker's Guide?"})

In [ ]:
response["response"]

AIMessage(content='The query is unrelated to the provided context and cannot be answered.')

In [ ]:
for context in response["context"]:
  print("Context:")
  print(context)
  print("----")

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the airspeed velocity of an unladen swallow?"})

In [ ]:
response["response"]

AIMessage(content='The query is unrelated to the provided context and cannot be answered.')

In [ ]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Where does Arthur Dent meet Marvin?"})

In [ ]:
response["response"]

AIMessage(content='The query is unrelated to the provided context and cannot be answered.')

####❓ Question #4:

Where does Arthur Dent meet Marvin?

> HINT: Use your RAG Chain to answer this question.

#### ✅ Answer:

"The query is unrelated to the provided context and cannot be answered."